In [ ]:
import numpy as np
import math
import torch
import einops
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
import pandas as pd
import matplotlib.pyplot as plt
import bottleneck as bn
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import analysis, utils
from sbi.inference import SNPE, simulate_for_sbi
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)
# import required modules
from sbi.utils.get_nn_models import posterior_nn
from sbi.neural_nets.embedding_nets import (
    FCEmbedding,
    CNNEmbedding,
    PermutationInvariantEmbedding
)
seed = 0
torch.manual_seed(seed)

In [ ]:
with open('your_path.pkl', 'rb') as f:
    posterior = torch.load(f)

In [ ]:
from matplotlib.lines import Line2D
#to compute the threshold plot for one joint posterior sample

# Generate the posterior samples
true_params = torch.tensor([1, 0.5], dtype=torch.float32).unsqueeze(0)
observation = Brownian_Motion_simulator(true_params).to('cuda')
posterior_samples_samples = posterior.sample((10000,), x=observation)

# Convert to a NumPy array
combined_samples = posterior_samples_samples.cpu().numpy()

# Extract log(m) and log(γ)
log_m_samples = combined_samples[:, 0]  # First column
log_gamma_samples = combined_samples[:, 1]  # Second column

# Compute 2D histogram and normalize
hist, x_edges, y_edges = np.histogram2d(
    log_m_samples, log_gamma_samples, bins=100, range=[[-1, 2], [-1, 2]]
)
hist_norm = hist / hist.sum()

# Compute cumulative sum and find the density threshold
hist_flat = hist_norm.flatten()
sorted_indices = np.argsort(hist_flat)[::-1]
sorted_hist_flat = hist_flat[sorted_indices]

def is_within_threshold(threshold):
    cumulative_sum = np.cumsum(sorted_hist_flat)
    threshold_index = np.searchsorted(cumulative_sum, threshold)
    density_high_threshold = sorted_hist_flat[threshold_index]
    return density_high_threshold

threshold = 0.01
step = 0.01
while threshold <= 1.0:
    density_high_threshold = is_within_threshold(threshold)
    mask = hist_norm.T >= density_high_threshold

    # Use histogram bin edges to find where `true_params` falls
    x_centers = (x_edges[:-1] + x_edges[1:]) / 2
    y_centers = (y_edges[:-1] + y_edges[1:]) / 2
    x_bin = np.digitize(true_params[0, 0].item(), x_centers) - 1
    y_bin = np.digitize(true_params[0, 1].item(), y_centers) - 1

    if mask[y_bin, x_bin]:  # Note: y_bin comes first in 2D arrays
        break
    threshold += step

# Create the plot
plt.figure(figsize=(8, 6))
plt.hist2d(log_m_samples, log_gamma_samples, bins=100, cmap='viridis', range=[[-1, 2], [-1, 2]])
plt.colorbar(label='ρ')

# Highlight the highest density regions using contour
X, Y = np.meshgrid(x_centers, y_centers)
contour = plt.contour(
    X, Y, hist_norm.T, levels=[density_high_threshold], colors='black', linewidths=1, linestyles='-'
)

# Add labels to the contour
#plt.clabel(contour, inline=False, fontsize=8, fmt={density_high_threshold: f'{threshold:.2f} ρ'})

# Add proxy artists for the legend
contour_legend = Line2D([0], [0], color='black', linewidth=1, linestyle='-', label=f'{threshold:.2f} ρ')
scatter_legend = Line2D([0], [0], marker='o', color='red', markersize=2, linestyle='None', label='$θ_0$')

# Plot the true parameter as a red dot
plt.scatter([true_params[0, 0].item()], [true_params[0, 1].item()], color='red', s=5, zorder=5)

# Add labels, title, and legend
plt.xlabel('log(m)')
plt.ylabel('log(γ)')
plt.legend(handles=[contour_legend, scatter_legend], loc='upper right')
plt.show()

In [ ]:
from matplotlib.lines import Line2D

# Generate the posterior samples
true_params = torch.tensor([1, 0.5], dtype=torch.float32).unsqueeze(0)
observation = Brownian_Motion_simulator(true_params).to('cuda')
posterior_samples_samples = posterior.sample((10000,), x=observation)

# Convert to a NumPy array
combined_samples = posterior_samples_samples.cpu().numpy()

# Extract log(m) and log(γ)
log_m_samples = combined_samples[:, 0]  # First column
log_gamma_samples = combined_samples[:, 1]  # Second column

# Compute 2D histogram and normalize
hist, x_edges, y_edges = np.histogram2d(
    log_m_samples, log_gamma_samples, bins=100, range=[[-1, 2], [-1, 2]]
)
hist_norm = hist / hist.sum()

# Compute cumulative sum and find the density threshold
hist_flat = hist_norm.flatten()
sorted_indices = np.argsort(hist_flat)[::-1]
sorted_hist_flat = hist_flat[sorted_indices]

def is_within_threshold(threshold):
    cumulative_sum = np.cumsum(sorted_hist_flat)
    threshold_index = np.searchsorted(cumulative_sum, threshold)
    density_high_threshold = sorted_hist_flat[threshold_index]
    return density_high_threshold

threshold = 0.01
step = 0.01
while threshold <= 1.0:
    density_high_threshold = is_within_threshold(threshold)
    mask = hist_norm.T >= density_high_threshold

    # Use histogram bin edges to find where `true_params` falls
    x_centers = (x_edges[:-1] + x_edges[1:]) / 2
    y_centers = (y_edges[:-1] + y_edges[1:]) / 2
    x_bin = np.digitize(true_params[0, 0].item(), x_centers) - 1
    y_bin = np.digitize(true_params[0, 1].item(), y_centers) - 1

    if mask[y_bin, x_bin]:  # Note: y_bin comes first in 2D arrays
        break
    threshold += step

# Create the plot
plt.figure(figsize=(8, 6))
plt.hist2d(log_m_samples, log_gamma_samples, bins=100, cmap='viridis', range=[[-1, 2], [-1, 2]])
plt.colorbar(label='ρ')

# Highlight the highest density regions using contour
X, Y = np.meshgrid(x_centers, y_centers)
contour = plt.contour(
    X, Y, hist_norm.T, levels=[density_high_threshold], colors='black', linewidths=1, linestyles='-'
)

# Add labels to the contour
#plt.clabel(contour, inline=False, fontsize=8, fmt={density_high_threshold: f'{threshold:.2f} ρ'})

# Add proxy artists for the legend
contour_legend = Line2D([0], [0], color='black', linewidth=1, linestyle='-', label=f'{threshold:.2f} ρ')
scatter_legend = Line2D([0], [0], marker='o', color='red', markersize=2, linestyle='None', label='$θ_0$')

# Plot the true parameter as a red dot
plt.scatter([true_params[0, 0].item()], [true_params[0, 1].item()], color='red', s=5, zorder=5)

# Add labels, title, and legend
plt.xlabel('log(m)')
plt.ylabel('log(γ)')
plt.legend(handles=[contour_legend, scatter_legend], loc='upper right')
plt.show()

In [ ]:
from matplotlib.lines import Line2D

# Generate the posterior samples
true_params = torch.tensor([1, 0.5], dtype=torch.float32).unsqueeze(0)
observation = Brownian_Motion_simulator(true_params).to('cuda')
posterior_samples_samples = posterior.sample((10000,), x=observation)

# Convert to a NumPy array
combined_samples = posterior_samples_samples.cpu().numpy()

# Extract log(m) and log(γ)
log_m_samples = combined_samples[:, 0]  # First column
log_gamma_samples = combined_samples[:, 1]  # Second column

# Compute 2D histogram and normalize
hist, x_edges, y_edges = np.histogram2d(
    log_m_samples, log_gamma_samples, bins=100, range=[[-1, 2], [-1, 2]]
)
hist_norm = hist / hist.sum()

# Compute cumulative sum and find the density threshold
hist_flat = hist_norm.flatten()
sorted_indices = np.argsort(hist_flat)[::-1]
sorted_hist_flat = hist_flat[sorted_indices]

def is_within_threshold(threshold):
    cumulative_sum = np.cumsum(sorted_hist_flat)
    threshold_index = np.searchsorted(cumulative_sum, threshold)
    density_high_threshold = sorted_hist_flat[threshold_index]
    return density_high_threshold

threshold = 0.01
step = 0.01
while threshold <= 1.0:
    density_high_threshold = is_within_threshold(threshold)
    mask = hist_norm.T >= density_high_threshold

    # Use histogram bin edges to find where `true_params` falls
    x_centers = (x_edges[:-1] + x_edges[1:]) / 2
    y_centers = (y_edges[:-1] + y_edges[1:]) / 2
    x_bin = np.digitize(true_params[0, 0].item(), x_centers) - 1
    y_bin = np.digitize(true_params[0, 1].item(), y_centers) - 1

    if mask[y_bin, x_bin]:  # Note: y_bin comes first in 2D arrays
        break
    threshold += step

# Create the plot
plt.figure(figsize=(8, 6))
plt.hist2d(log_m_samples, log_gamma_samples, bins=100, cmap='viridis', range=[[-1, 2], [-1, 2]])
plt.colorbar(label='ρ')

# Highlight the highest density regions using contour
X, Y = np.meshgrid(x_centers, y_centers)
contour = plt.contour(
    X, Y, hist_norm.T, levels=[density_high_threshold], colors='black', linewidths=1, linestyles='-'
)

# Add labels to the contour
#plt.clabel(contour, inline=False, fontsize=8, fmt={density_high_threshold: f'{threshold:.2f} ρ'})

# Add proxy artists for the legend
contour_legend = Line2D([0], [0], color='black', linewidth=1, linestyle='-', label=f'{threshold:.2f} ρ')
scatter_legend = Line2D([0], [0], marker='o', color='red', markersize=2, linestyle='None', label='$θ_0$')

# Plot the true parameter as a red dot
plt.scatter([true_params[0, 0].item()], [true_params[0, 1].item()], color='red', s=5, zorder=5)

# Add labels, title, and legend
plt.xlabel('log(m)')
plt.ylabel('log(γ)')
plt.legend(handles=[contour_legend, scatter_legend], loc='upper right')
plt.show()